# Loads and Imports

In [1]:
%load_ext autoreload
%autoreload 2
%load_ext Cython
%reload_ext Cython

In [2]:
#built-in
import pandas as pd
import numpy as np
import time
import datetime as dt
from pathlib import Path
import time
from tqdm import tqdm
from sklearn.neighbors import NearestNeighbors
import pyarrow.parquet as pq
import seaborn as sns
import sys
import boto3

path = str(Path.home()) + '/rpi-canary-eval/temp'
git_path = str(Path.home()) + '/rpi-canary-eval'
sys.path.append(git_path + '/modules')


#custom
import data_process
from data_functions import make_date_to_MYR
from diabetes_onset import( process_canary,
get_tc, in_canary, diabet_onset, not_first_month_diabetes, 
                           match_data_per_month, 
                           filter_first_month_diabetes,
                           matched_population, 
                           propensity_func,
                           nearest_neighbor_func, 
                           random_matching_func, agglomerative )

from path import path_dict, col_seq_cc, col_seq_elx, col_seq_cc_all

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
np.random.seed(seed = 0)

/home/nneehal/.local/lib/python3.6/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# Download and Import Latent Space data and corresponding Original Data from S3

In [3]:
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/GEORGE_202108_preg/KNN/treated0.p ../../temp/odata_knn/g_treated0_o.p
# !aws s3 cp s3://cdphp-s3-us-e-p-pond/rpi/nneehal/DATA/GEORGE_202108_preg/KNN/match0.p ../../temp/odata_knn/g_match0_o.p

In [4]:
import pickle
with open(path+'/odata_knn/g_match0_o.p', 'rb') as fp:
    controls_sick_original = pickle.load(fp)
with open(path+'/odata_knn/g_treated0_o.p', 'rb') as fp:
    treated_sick_original = pickle.load(fp)

# Sort and Filter

In [6]:
controls_orig = controls_sick_original[controls_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])
treated_orig = treated_sick_original[treated_sick_original.PERS_ID>"0"].sort_values(by=["PERS_ID", "MYR"])

## Evaluation Quality Match

## Metric = ASMD

$$d = |\frac{(\bar{x_t}-\bar{x_c})}{\sqrt{\frac{s_t^2 + s_c^2}{2}}}| $$

In [7]:
def ASMD_cont_calculate(tf, cf):
    t_mean = np.mean(tf.values)
    c_mean = np.mean(cf.values)
    t_var = np.var(tf.values)
    c_var = np.var(cf.values)
    d = np.abs((t_mean-c_mean)/np.sqrt((t_var+c_var)/2))
    return d

In [8]:
def ASMD_binary_calculate(tf, cf):
    pt = np.mean(tf.values)
    pc = np.mean(cf.values)
    d = np.abs((pt-pc)/(pt*(1-pt)+pc*(1-pc)))
    return d
    

In [9]:
def ASMD_final(cols, col_cat, cdf, tdf):
    ASMD = pd.DataFrame()
    cf = cdf[cols]
    tf = tdf[cols]
    for i, (col_cat_each, col_name_each) in enumerate(zip(col_cat, cols)):
        if col_cat_each == 'c':
            d = ASMD_cont_calculate(tf[col_name_each], cf[col_name_each])
        else:
            d = ASMD_binary_calculate(tf[col_name_each], cf[col_name_each])
        ASMD.loc[0, col_name_each] = d 
    return ASMD
    

### Original Space with George's Features only

In [14]:
from itertools import repeat

In [15]:
george_cols = ['AGE_AT_MIDMONTH', 'MBR_GNDR', 'ALLOW_AMT','CC_TOBACCO',  'CC_OBESITY',
                   'CC_HYPERTENSION', 'CC_HYPOTHYROID']
george_cols_cat = ['c', 'b', 'c', 'b', 'b', 'b', 'b']

In [16]:
ASMD = ASMD_final(george_cols, george_cols_cat, controls_orig, treated_orig)
ASMD

AGE_AT_MIDMONTH  MBR_GNDR  ALLOW_AMT  CC_TOBACCO  CC_OBESITY  CC_HYPERTENSION  CC_HYPOTHYROID
0         0.043853  0.035524   0.003537    0.028437    0.088352         0.008186        0.114922

In [26]:
medicaid_mask = treated_orig.MEDICAID_LOB.str.contains(r'.*Medicaid-.*')
treated_orig['LOB'] = 1
treated_orig.loc[medicaid_mask, 'LOB'] = 0
treated_orig['MEDICAID_LOB'] = treated_orig['LOB']
treated_orig = treated_orig.drop(['LOB'], axis=1)

In [27]:
medicaid_mask = controls_orig.MEDICAID_LOB.str.contains(r'.*Medicaid-.*')
controls_orig['LOB'] = 1
controls_orig.loc[medicaid_mask, 'LOB'] = 0
controls_orig['MEDICAID_LOB'] = controls_orig['LOB']
controls_orig = controls_orig.drop(['LOB'], axis=1)

### Original Space with all the features

In [28]:
original_cols = controls_orig.columns.to_list()[2:]

In [29]:
l1 = ['c', 'b', 'b', 'c', 'c', 'c']

given_value ='b'

orig_CC_cols_cat=[]
orig_CC_cols_cat.extend(repeat(given_value,len(original_cols)-len(l1)))

original_cols_cat = l1 + orig_CC_cols_cat

In [30]:
ASMD = ASMD_final(original_cols, original_cols_cat, controls_orig, treated_orig)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in double_scalars
  after removing the cwd from sys.path.


In [38]:
ASMD.T

0
MBR_GNDR                    0.028064
AGE_AT_MIDMONTH             0.000107
ALLOW_AMT                   0.000012
MEDICAID_LOB                0.041988
ALLOW_ER                    0.006507
ALLOW_IP                    0.010675
CC_ADHD                     0.082652
CC_ALZHEIMER                1.000519
CC_ALZHEIMER_PLUS           0.717178
CC_AMI                      0.659851
CC_ANEMIA                   0.011529
CC_ANXIETY_DISORDER         0.029898
CC_ARTHRITIS                0.022813
CC_ASTHMA                   0.030225
CC_ATRIAL_FIB               0.098722
CC_AUTISM                   1.002090
CC_BIPOLAR                  0.034711
CC_BPH                      0.075733
CC_BREAST_CANCER            0.060694
CC_CATARACT                 0.006112
CC_CEREBRAL_PALSY           0.029083
CC_CHRONIC_KIDNEY           0.045678
CC_COLORECTAL_CANCER        0.321802
CC_COPD                     0.067879
CC_CYSTIC_FIBROSIS          0.336840
CC_DEPRESSION               0.007251
CC_DEPRESSIVE_DISORDERS     0.175920
CC_DEVELOP_DELAYS           0.357882
CC_DIABETES                 0.052111
CC_DISAB_DX_CNT             0.074753
CC_DX_CNT                   0.000170
CC_ENDOMETRIAL_CANCER       0.113575
CC_EPILEPSY                 0.222812
CC_FIBROMYALGIA             0.090274
CC_GLAUCOMA                 0.047229
CC_HEARING_IMPAIR           0.015627
CC_HEART_FAILURE            0.020414
CC_HEPATITIS_A              1.000061
CC_HEPATITIS_B_ACUTE        0.264935
CC_HEPATITIS_B_CHRONIC      0.104325
CC_HEPATITIS_C_ACUTE             NaN
CC_HEPATITIS_C_CHRONIC      0.532420
CC_HEPATITIS_C_UNSPECIFIED  0.241391
CC_HEPATITIS_D                   NaN
CC_HEPATITIS_E                   NaN
CC_HEPATITIS_GEN            0.321759
CC_HIP_FRACTURE             1.000234
CC_HYPERLIPIDEMIA           0.021242
CC_HYPERTENSION             0.008186
CC_HYPOTHYROID              0.114922
CC_INT_DISAB                0.636189
CC_ISCHEMIC                 0.027863
CC_LEARN_DISAB              0.035427
CC_LEUKEMIAS                0.119081
CC_LIVER                    0.005491
CC_LUNG_CANCER              0.087776
CC_MIGRAINE                 0.072186
CC_MOBILITY_IMPAIR          0.035268
CC_MULTIPLE_SCLEROSIS       0.071641
CC_MUSCULAR_DYSTROPHY       1.000397
CC_OBESITY                  0.088352
CC_OSTEOPOROSIS             0.187822
CC_PERIPHERAL               0.094709
CC_PERSONALITY              0.057588
CC_PRESSURE                 0.171792
CC_PROSTATE_CANCER          0.008715
CC_PTSD                     0.017471
CC_SCHIZOPHRENIA            0.198024
CC_SCHIZOPHRENIA_OTHER      0.280221
CC_SPINAL_CORD              0.298276
CC_SPINA_BIFIDA             0.079130
CC_STROKE                   0.011634
CC_TOBACCO                  0.028437
CC_TRAUMATIC_BRAIN          0.196650
CC_VISUAL_IMPAIR            0.630121
PREGNANCY                   0.315309
IP_BIN                      0.128381
ER_BIN                      0.055567
IP_BIN2                     0.121215
ER_BIN2                     0.036912
IP_C                        0.026411
ER_C                        0.054184

In [37]:
(ASMD.loc[0,:]>0.1).sum()

31